# Detect and blur faces in video using Amazon Rekognition Video

***
This notebook provides a walkthrough of [face detection API](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceDetection.html) in Amazon Rekognition Video to identify faces in a stored video.
***

## Initialization

Installing required libraries:   
-**opencv-python** : pre-built CPU-only OpenCV packages for Python. OpenCV is a library of programming functions mainly aimed at real-time computer vision.  
-**ffmpeg-python** : python bindings for FFmpeg. FFmpeg is a complete, cross-platform solution to record, convert and stream audio and video.   
-**moviepy** :a Python library for video editing: cutting, concatenations, title insertions, video compositing (a.k.a. non-linear editing), video processing, and creation of custom effects. 

In [ ]:
!pip install opencv-python boto3 botocore ffmpeg-python moviepy --upgrade -q

In [ ]:
import boto3
import botocore

In [ ]:
client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account:{account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

In [ ]:
# S3 bucket that will be the location from where we will read images/videos

bucket = 'dml102-20221003-sagemaker-bucket-{region}-{account}'.format(region = region, account = account)

In [ ]:
# copy video from local EBS storage to S3 bucket

!aws s3 cp videos/people-walking.mp4 s3://$bucket/rekognition-video-demo/people-walking.mp4

## Detect faces in video
***

Explore the code in blur_faces/

In [ ]:
from blur_faces.rekognition import get_timestamps_and_faces
from blur_faces.rekognition import boto3_client, start_face_detection
from blur_faces.rekognition import wait_for_completion
from blur_faces.video_processor import apply_faces_to_video, integrate_audio

Amazon Rekognition reads the video from an S3 bucket.

In [ ]:
video = 'rekognition-video-demo/people-walking.mp4'
local_path_to_video = 'videos/people-walking.mp4'
local_output = 'videos/output.mp4'

1. Calling the start face detection API. Async
```
def start_face_detection(bucket, video, size, reko_client=None):
    assert check_format_and_size(video, size)
    if reko_client == None:
        reko_client = boto3.client('rekognition')
    response = reko_client.start_face_detection(Video={'S3Object': {'Bucket': bucket, 'Name': video}})
    return response['JobId']
```
2. Checking the progress of the job, by calling the get_face_detection(JobId=job_id) API every 30sec
```
def wait_for_completion(job_id, wait_time_in_s=30, reko_client=None):
    if reko_client == None:
        reko_client = boto3.client('rekognition')
    response = reko_client.get_face_detection(JobId=job_id)
    while (response['JobStatus'] == 'IN_PROGRESS'):
        print('.', end='')
        time.sleep(wait_time_in_s)
        response = reko_client.get_face_detection(JobId=job_id)
    print('Complete')
    return response  
```
3. get timestamps
4. apply faces to video

In [ ]:
reko = boto3_client()
job_id = start_face_detection(bucket, video, 1, reko)
response = wait_for_completion(job_id, reko_client=reko)
timestamps=get_timestamps_and_faces(response, job_id, reko)
apply_faces_to_video(timestamps, local_path_to_video, local_output, response["VideoMetadata"])
# integrate_audio('videos/video-test.mp4', 'videos/output.mp4')

![Serverless architecture](img/blur-faces-arch.png)

1. The event flow starts at the moment of the video ingestion into Amazon S3. Amazon Rekognition Video supports MPEG-4 and MOV file formats, encoded using the H.264 codec.
2. After the video file has been stored into Amazon S3, it automatically kicks-off an event triggering a Lambda function.
3. The Lambda function uses the video’s attributes (name and location on Amazon S3) to start the face detection job on Amazon Rekognition through an API call.
4. The same Lambda function then starts the Step Functions state machine, forwarding the video’s attributes and the Amazon Rekognition job ID.
5. The Step Functions workflow starts with a Lambda function waiting for the Amazon Rekognition job to be finished. Once it’s done, another Lambda function gets the results from Amazon Rekognition.
6. Finally, a Lambda function with Container Image Support fetches its Docker image, which supports OpenCV from Amazon ECR, blurs the faces detected by Amazon Rekognition, and temporarily stores the output video locally.
7. Then, the blurred video is put into the output S3 bucket and removed from local files.